# 디렉토리 설정 

In [ ]:
!unzip /content/drive/MyDrive/DataSet/KYR_B_imaging.zip
!cp '/content/drive/MyDrive/Colab Notebooks/CT분류프로젝트/Custom_Py/init.py' ./

In [ ]:
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr3' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr2 74' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr5' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr1' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr7' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr8' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr4' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/nswr1' '/content/KYR_B_imaging' 
!cp -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱/swr6' '/content/KYR_B_imaging' 
!rm -r '/content/KYR_B_imaging/20201224 DL_MRI_SNSB_임승욱'

# 초기작업

## 기본 DF 생성 

In [ ]:
import os 
import cv2 
import pandas as pd 
from glob import glob
from tqdm import tqdm
from init import init #초기 설정용 커스텀 패키지 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.pyplot as cm 
import warnings
warnings.filterwarnings('ignore')

###snsb_df 생성 

In [ ]:
columns = ['성명',
 '병록번호',
 '만나이',
 '검사일',
 '교육년수',
 'SVLT_recall_total_score_z',
 'SVLT_Delayed_recall_z',
 'SVLT_recognition_score_z',
 'RCFT_immediate_recall_z',
 'RCFT_delayed_recall_z',
 'RCFT_recognition_score_z',
 'K_MMSE_total_score_z',
 'SNSB_II_Domain_Attention_z',
 'SNSB_II_Domain_Language_z',
 'SNSB_II_Domain_Visuospatial_z',
 'SNSB_II_Domain_Memory_z',
 'SNSB_II_Domain_Frontal_z']

snsb_df = pd.read_csv('/content/drive/MyDrive/DataSet/202202_김예림/SNSB_integerated.csv')
snsb_df = snsb_df.drop([2098,2591]) #병록번호 없는 행 제거 <- 결측치 
snsb_df['병록번호'] = snsb_df['병록번호'].apply(lambda x : str(int(x)))

snsb_df = snsb_df[columns]

###dir_df 생성 

In [ ]:
#폴더 디렉토리
folder_dir = glob('/content/KYR_B_imaging/*')
dir_df = init.dir_df(folder_dir)
dir_df = dir_df.sort_values(by=['dir'])

### 병록번호- key 값 보정

In [ ]:
target_length = [4,5,6,7,8] #<- 수정해야 하는 병록번호 갯수들 

dir_df,snsb_df = init.key_mismatch(dir_df,snsb_df,target_length)
# plt.hist(snsb_df['병록번호'].map(len))
# plt.show()

#에러값들 제외 
errors = ['000758836-1' '030338456-1' '040145556-1' '050236926-1' '090374796-1'
 '100402746-1' '120010356-1' '870133669-1' '920033543-1' '930236559-1'
 '930257597-1' '970682600-1']

dir_df = dir_df.drop(np.where(dir_df['key'].apply(lambda x : x in errors))[0])

##이미지 장수 조절


### Key 별 이미지 갯수 리스트 생성 

Key 별 이미지 장수 확인 

In [ ]:
#이미지 장수 연산 
keys = list(set(dir_df['key'])) 
images_length = pd.DataFrame(keys)[0].apply(lambda x: init.check_images(dir_df,x)) #키 값별로 이미지의 장수들 계산 
# images_length.value_counts()

length_df = pd.DataFrame([keys,images_length]).T
length_df.columns = ['key','image_length']
length_df['image_length']  = length_df['image_length'].map(int)

#Standard를 기준으로 less over 분할 
standard = 19 # 통일 시킬 이미지 장 수 기준 

# 조정이 필요한 장 수 초과하는 것들만 추림 
length_df_over = length_df[length_df['image_length']>standard]

#19장 보다 적은 경우 
# length_df_less = length_df[length_df['image_length']<standard]

### new_dir_df 생성 

In [ ]:
new_dir_df = pd.DataFrame(np.zeros(1606*19*2).reshape(-1,2))
new_dir_df.columns = dir_df.columns

for index in tqdm(range(1606)):
  new_dir_df.iloc[index*19:(index+1)*19,:]  = dir_df.loc[init.droped_indexes(dir_df,length_df_over,index,standard=19)]

100%|██████████| 1606/1606 [00:07<00:00, 211.37it/s]


## z score 임베딩 생성

In [ ]:
Embedding_columns = snsb_df.columns[4:]

for column in Embedding_columns:
  snsb_df[f'E_{column}'] = snsb_df[column].map(init.zscore_Embedding)

#정렬 
snsb_df = snsb_df.sort_values(by=['병록번호'])
new_dir_df = new_dir_df.sort_values(by=['key'])

csvs = snsb_df[snsb_df['병록번호'].apply(lambda x : x in key_lists)].sort_values(by=['병록번호'])
csvs = csvs.sort_values(by=['성명'])
csvs = csvs.drop_duplicates(['병록번호'], keep = 'first')
csvs = csvs.sort_values(['병록번호'])

# 데이터 로드 

## 제너레이터 사전 설정 

In [ ]:
input_columns = ['만나이','교육년수']
output_columns = ['E_SVLT_recall_total_score_z', 'E_SVLT_Delayed_recall_z',
       'E_SVLT_recognition_score_z', 'E_RCFT_immediate_recall_z',
       'E_RCFT_delayed_recall_z', 'E_RCFT_recognition_score_z',
       'E_K_MMSE_total_score_z', 'E_SNSB_II_Domain_Attention_z',
       'E_SNSB_II_Domain_Language_z', 'E_SNSB_II_Domain_Visuospatial_z',
       'E_SNSB_II_Domain_Memory_z', 'E_SNSB_II_Domain_Frontal_z']

csvs_max = np.array([90,18])
csvs_min = np.array([45,0])

In [ ]:
#키값으로 이미지 dir 검색 
def temp_dir_df(new_dir_df,key):
  temp_df = new_dir_df[new_dir_df['key']==key]
  return temp_df

#더미 넘파이에 한개씩 넣음 -> 19개 넣어서 19,512,512,3 shape 만듬 
def read_imgs(temp_df):
  temp_imgs = []
  for dir_index in range(19):
    temp_img = cv2.imread(temp_df['dir'].iloc[dir_index])
    temp_img = cv2.resize(temp_img,dsize= (256,256))
    # temp_img = cv2.cvtColor(temp_img,cv2.COLOR_BGR2GRAY)
    temp_imgs.append(temp_img)
  temp_imgs = np.array(temp_imgs)
  return temp_imgs
  
# 위 두개 통합 
def make_temp_imgs(new_dir_df,key):
  temp_df = temp_dir_df(new_dir_df,key)
  temp_imgs = read_imgs(temp_df)
  return temp_imgs

In [ ]:
#파라미터 
/

## 제너레이터

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence
import math


class DataGenerator(Sequence):
  def __init__(self,
                 key_lists,
                 new_dir_df,
                 csvs,
                 batch_size= 1,
                 validation_split = 0.1,
                 test_split = 0.1,
                 augmentation: bool = False,
                 shuffle: bool = False,
                 rescale:bool = True,
               indexes = None) -> None:
        self.key_lists = key_lists 
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.validation_split = validation_split
        self.augmentation = augmentation
        self.rescale =rescale
        self.new_dir_df = new_dir_df
        self.csvs = csvs
        self.indexes, self.valid_indexes, self.test_indexes = self._prepare_indexes(validation_split, test_split, indexes)


          # shuffle for first epoch
        if self.shuffle:
            self.shuffle_data()

  def __len__(self):
    return math.ceil(len(self.key_lists) / self.batch_size)
  
  def __getitem__(self,index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    return self._generate_data(indexes)

  def on_epoch_end(self):
        self._shuffle_index()
  
  def _prepare_indexes(self, validation_split, test_split, indexes):
        if indexes is not None:
            return indexes, None, None
        
        data_count = len(self.new_dir_df)
        all_index = np.arange(data_count)

        if validation_split+test_split==0.0:
            return all_index, None, None
            
        valid_i = int(len(all_index)*(1-validation_split-test_split))
        test_i = int(len(all_index)*(1-test_split))

        return all_index[:valid_i], all_index[valid_i:test_i], all_index[test_i:]

  def _shuffle_index(self):
      if self.shuffle == True:
          np.random.shuffle(self.indexes)
  
  def _generate_data(self, indexes): #indexes = [0:1]
     csvs_input = self.csvs[input_columns].iloc[indexes]
     csvs_output = self.csvs[output_columns].iloc[indexes]

     image_batch = self.key_lists[indexes]
     imgs = []
     for key_index in range(len(image_batch)):
       key = self.key_lists[key_index]
       temp_imgs = make_temp_imgs(self.new_dir_df,key)
       imgs.append(temp_imgs)

     csvs_input = np.array(csvs_input)
     csvs_input = (csvs_input - csvs_min)/(csvs_max - csvs_min)
     csvs_output = np.array(csvs_output)
     imgs=  np.array(imgs)
     # return [imgs, csvs_input], csvs_output
     return imgs, csvs_output

        
  def subset(self, type):
    if type=="train":
        return DataGenerator(self.key_lists,self.new_dir_df,self.csvs,indexes=self.indexes)
    elif type=="validation" or type=="valid":
        return DataGenerator(self.key_lists,self.new_dir_df,self.csvs, indexes=self.valid_indexes)
    elif type=="test":
        return DataGenerator(self.key_lists,self.new_dir_df,self.csvs, indexes=self.test_indexes)
    else:
        print("invalid type {}".format(type))

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence
import math


class DataGenerator(Sequence):
  def __init__(self,
                 key_lists,
                 new_dir_df,
                 snsb_df,
                 batch_size: int,
                 augmentation: bool = False,
                 shuffle: bool = False,
                 rescale:bool = True) -> None:
        self.key_lists = key_lists 
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.rescale =rescale
        self.new_dir_df = new_dir_df
        self.csvs = csvs

          # shuffle for first epoch
        if self.shuffle:
            self.shuffle_data()

  def __len__(self):
    return math.ceil(len(self.key_lists) / self.batch_size)

  def __getitem__(self, index):
    csvs_input = csvs[input_columns].iloc[index*self.batch_size:(index+1):self.batch_size,:]
    csvs_output = csvs[output_columns].iloc[index*self.batch_size:(index+1):self.batch_size,:]

    image_batch = self.key_lists[index * self.batch_size:(index + 1) * self.batch_size]
    imgs = []
    for key_index in range(len(image_batch)):
      key = self.key_lists[key_index]
      temp_imgs = make_temp_imgs(self.new_dir_df,key)
      imgs.append(temp_imgs)

    csvs_input = np.array(csvs_input)
    csvs_input = (csvs_input - csvs_min)/(csvs_max - csvs_min)
    csvs_output = np.array(csvs_output)
    imgs=  np.array(imgs)
    # return [imgs, csvs_input], csvs_output
    return imgs, csvs_output

In [ ]:
data_generator = DataGenerator(key_lists,new_dir_df,csvs,validation_split=0.2,test_split=0.1)
train_data_generator = data_generator.subset("train")
valid_data_generator = data_generator.subset("valid")
test_data_generator = data_generator.subset("test")

# 모델

In [ ]:
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Bidirectional, LSTM, GRU
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Dense,Flatten,Input,Conv3D,MaxPooling3D
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.applications import resnet50, efficientnet
from tensorflow.keras.applications import ResNet50, EfficientNetB0

In [ ]:
from tensorflow.python import keras
input = Input((19,256,256,3))
x = BatchNormalization()(input)
x = Conv3D(256,kernel_size=(3,3,3),padding='same',activation='relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)

x = Conv3D(128,kernel_size=(3,3,3),padding='same',activation='relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)

x = Conv3D(64,kernel_size=(3,3,3),padding='same',activation='relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)

x = Conv3D(64,kernel_size=(3,3,3),padding='same',activation='relu')(x)
x = MaxPooling3D((2,2,2))(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(16,activation = 'relu')(x)
x = BatchNormalization()(x)
output = Dense(12,activation = 'softmax')(x)

model = Model(input,output)
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_data_generator,validation_data = valid_data_generator, epochs=10,verbose=1)

Epoch 1/10
 943/1606 [================>.............] - ETA: 3:27 - loss: 14.6542 - accuracy: 0.1092

In [ ]:
# copy from https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e
# loss 실시간 출력 
from IPython.display import clear_output
from tensorflow.keras.callbacks import Callback

class PlotLosses(Callback):

  def on_train_begin(self, logs={}):
    self.epochs = []
    self.losses = []
    self.val_losses = []
    self.logs = []
    self.fig = plt.figure()


  def on_epoch_end(self, epoch, logs={}):

    self.epochs.append(epoch)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))

    clear_output(wait=True)
    plt.plot(self.epochs, self.losses, label="loss")
    plt.plot(self.epochs, self.val_losses, label="val_loss")
    plt.legend()
    plt.show();
    print("loss = ", self.losses[-1], ", val_loss = ", self.val_losses[-1])

#call backs 선언
plot_losses = PlotLosses()

from tensorflow.keras.callbacks import ModelCheckpoint #<- model 저장 객체, best_model.h5라는 모델 이 저장 됨 
model_check_point = ModelCheckpoint(
    'best_model.h5', 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True)


from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링 대상, 이걸 기준으로 멈춤 
    verbose=1,
    patience=50)         # 중지까지의 여유분

from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', # 모니터링 대상
    patience=25,        # 대상ㅃ 기간동안 유지
    factor=0.2,         # 줄이는 양                              
    min_learning_rate=0.00001)     # 최소 학습율

callbacks = [model_check_point, plot_losses, reduce_lr]